# Data Cleaning for Gender IDEAL

Before committing, please re-run the kernel with clear any output to avoid any merge issues with jupyter and github.

## Imports

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
#import altair as alt

pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

Leadership Demographics Category: Scoring
Leadership Weight: 7.5

In [2]:
dfl = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/leader_clean_final.csv", index_col=0)
dfl

,Organization,review_status,proportion_local_div,3A,3B,3C,3D,3E,3F,3G,3H,3I,3J,4A,4B,4C,4D,4E,4F,4G,4H,4I,5A,5B,5C,5D,5E,5F,5G,5H,5I,6A,6B,6C,6D,6E,6F,6G,6H,6I,7A,7B,7C,7D,7E,7F,7G,7H,7I,8A,8B,8C,8D,8E,8F,8G,8H,8I,8J,9A,9B,9C,9D,9E,9F,9G,10,benchmark_group,number_of_employees,number_of_years,workforce,9H
0,Company 1A,In progress,Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group),8.0,100.0,13.0,13.0,13.0,26.0,26.0,13.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,100.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,C-Suite,Board/Advisory Bodies,Senior Management,Management,Mentorship Programs,Internal Promotions,NaN,NaN,Finance,Fewer than 100 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN
1,Company 1B,No,NaN,216.0,38.0,1.0,1.0,6.0,0.0,0.0,1.0,0.0,0.0,10.0,4.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finance,100-249 Employees,15 or more years,At least 80% of employees are Salaried,NaN
2,Company 1C,No,NaN,30.0,17.0,3.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Hourly,NaN
3,Company 1D,No,NaN,35.0,45.0,12.0,6.0,0.0,6.0,0.0,30.0,50.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,0.0,75.0,25.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,Board/Advisory Bodies,NaN,NaN,NaN,NaN,NaN,NaN,Technology,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,NaN
4,Company 1E,No,NaN,83.0,70.0,9.0,18.0,4.0,NaN,7.0,NaN,NaN,NaN,76.0,6.0,9.0,6.0,NaN,3.0,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Internal Promotions,NaN,NaN,NonProfit,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,NaN
5,Company 1F,No,NaN,7.0,43.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Management,NaN,NaN,NaN,NaN,Finance,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,NaN
6,Company 1G,No,NaN,269.0,40.0,3.0,4.0,6.0,1.0,2.0,NaN,NaN,NaN,41.0,3.0,2.0,6.0,1.0,0.0,NaN,NaN,NaN,30.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,NaN,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,NaN
7,Company 1H,No,NaN,55.0,90.0,10.0,10.0,5.0,NaN,10.0,0.0,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,Technology,250-999 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN
8,Company 1I,No,NaN,57.0,73.0,5.0,3.0,5.0,0.0,2.0,NaN,4.0,0.0,52.0,0.0,0.0,5.0,0.0,0.0,NaN,0.0,0.0,67.0,8.0,0.0,8.0,0.0,0.0,NaN,NaN,0.0,33.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior Management,NaN,NaN,NaN,NaN,Nearing goals (66-99% targets met),NonProfit,Fewer than 100 Employees,5-14 y

In [3]:
# Questions 1, 2, 9, and 10 have equal weight of 25% of the Leadership score.
w1 = 0.25
w2 = 0.25
w9 = 0.25
w10 = 0.25
w = [w1, w2, w9, w10]

leader_weight = 7.5

# For each Company row, generate score by weights determined above.
dfl = dfl[['Organization', 'review_status', 'proportion_local_div', '9A', '9B', '9C', '9D', '9E', '9F', '9G', '10']].copy()
dfl

# Function to apply to each row:
def compute_score(row):
    
    # Q9:
    # No = 0% 
    # In Progress = 50%
    # Yes = 100%
    # No Sum
    
    q1 = row['review_status']
    if q1 == 'No':
        s1 = 0
    elif q1 == 'In progress':
        s1 = 0.5
    elif q1 == 'Yes':
        s1 = 1
    else:
        s1 = 0
        
    # Q2: 
    # Signficant disparities (>10%) = 33%
    # Modest disparities (<=10%) = 66%
    # Minimal to no disparities (<=5%) = 100%
    # No/N/A = 0%
    # No Sum
    
    q2 = row['proportion_local_div']
    if q2 == """Significant disparities
    (ie - one or more groups that represent
    >10% population not represented in our workforce)""":
        s2 = 0.33
    elif q2 == """Modest disparities
    (ie - the differential between our workforce
    and the population across different groups is 10% or less per group)""":
        s2 = 0.66
    elif q2 == """Minimal to no disparities
    (ie - the differential between our workforce 
    and the population across different groups is 5% or less per group)""":
        s2 = 1
    else: # N/A or No
        s2 = 0  

    # Q9:
    # C-Suite = 50%
    # Board/Advisory Bodies = 50%
    # Management = 50%
    # Mentorship Programs = 50%
    # Promotions = 50%
    # N/A or No = 0%
    # Multi-pick/ Responses sum; Cap at 100%"
    s9 = 0
    if row['9A'] == 'C-Suite':
        s9 += 0.5
    if row['9B'] == 'Board/Advisory Bodies':
        s9 += 0.5
    if row['9C'] == 'Senior Management':
        s9 += 0.5
    if row['9D'] == 'Management':
        s9 += 0.5
    if row['9E'] == 'Mentorship Programs':
        s9 += 0.5
    if row['9F'] == 'Internal Promotions':
        s9 += 0.5
    #9G excluded since all responses can only be No or NaN
    if s9 > 1:
        s9 = 1
        
    # Q10:
    # N/A - No targets set = 0%
    # Behind goals (<66% or fewer targets met) = 50%
    # Nearing goals (66-99% targets met) = 100%
    # Goals met = 100%
    # Goals exceeded = 100%
    # No Sum
    
    q10 = row['10']
    if q2 == 'Behind goals (<66% or fewer targets met)':
        s10 = 0.5
    elif q10 == 'Nearing goals (66-99% targets met)' or q10 == 'Goals met' or q10 == 'Goals exceeded': 
        s10 = 1
    else: #no targets set or NaN
        s10 = 0
    
    s = [s1, s2, s9, s10]
    
    #preliminary score
    return sum(map(lambda a, b: a * b, s, w))

dfl.loc[:,'leader_prelim_score'] = dfl.apply(compute_score, axis = 1)
dfl.loc[:,'leader_weighted_score'] = dfl['leader_prelim_score'] * leader_weight
dfl

,Organization,review_status,proportion_local_div,9A,9B,9C,9D,9E,9F,9G,10,leader_prelim_score,leader_weighted_score
0,Company 1A,In progress,Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group),C-Suite,Board/Advisory Bodies,Senior Management,Management,Mentorship Programs,Internal Promotions,NaN,NaN,0.375,2.8125
1,Company 1B,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0000
2,Company 1C,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0000
3,Company 1D,No,NaN,NaN,Board/Advisory Bodies,NaN,NaN,NaN,NaN,NaN,NaN,0.125,0.9375
4,Company 1E,No,NaN,NaN,NaN,NaN,NaN,NaN,Internal Promotions,NaN,NaN,0.125,0.9375
5,Company 1F,No,NaN,NaN,NaN,NaN,Management,NaN,NaN,NaN,NaN,0.125,0.9375
6,Company 1G,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,0.000,0.0000
7,Company 1H,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,0.000,0.0000
8,Company 1I,No,NaN,NaN,NaN,Senior Management,NaN,NaN,NaN,NaN,Nearing goals (66-99% targets met),0.375,2.8125
9,Company 1J,In progress,Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group),NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,0.125,0.9375


In [4]:
#Write out to csv
dfl.to_csv("C:/Users/Admin/Desktop/gender-ideal-main/scores/leader_scores_clean.csv")

Vision Category: Scoring
Vision Weight: 10

In [5]:
dfv = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/data_clean/vision_clean_final.csv", index_col=0)

# Reverse indices so that Company 1A is listed first.
dfv.index = dfv.index.values[::-1]
dfv = dfv[::-1]
dfv

dfv

,Organization,number_of_employees,number_of_years,workforce,Q9_A,Q9_B,Q9_C,Q9_D,Q9_E,Q9_F,Q9_G,Q9_H,Q9_I,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11_A,Q11_B,Q11_C,Q11_D,Q11_E,Q11_F,Q12_A,Q12_B,Q12_C,Q12_D,Q12_E,Q12_F,Q13_A,Q13_B,Q13_C,Q14,Q14_Other,Q15_A,Q15_B,Q15_C,Q15_D,Q15_E,Q15_F,Q15_G,benchmark_group,Q11_G,Q12_G
0,Company 1A,Fewer than 100 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,"Yes, internal statement made to all employees","Yes, official public/external statement made about organization's commitment to gender equity",No EDI report is published,NaN,Womens Empowerment Principles (UN),NaN,NaN,NaN,NaN,NaN,Gender Equity Now,Finance,NaN,NaN
1,Company 1B,100-249 Employees,15 or more years,At least 80% of employees are Salaried,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,All Hiring Managers,Human Resources Dept,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Finance,NaN,NaN
2,Company 1C,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Hourly,NaN,NaN,NaN,"Women of color, including all races and ethnicities",NaN,NaN,NaN,NaN,NaN,NaN,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,Training of all employees who have a direct impact on meeting targets,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,Other,"Prepare report on work on EDI, no gender performance goals specifically called out.",NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
3,Company 1D,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conducted a demographic analysis of the current workforce,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Technology,NaN,NaN
4,Company 1E,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,Human Resources Dept,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NonProfit,NaN,NaN
5,Company 1F,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,Publish an annual EDI report that includes performance data on gender equity,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Finance,NaN,NaN
6,Company 1G,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Sal

In [6]:
# Questions 1, 2, 9, and 10 have equal weight of 25% of the Leadership score.
w9 = 0.10
w10 = 0.15
w11 = 0.20
w12 = 0.25
w13 = 0.20
w14 = 0.10
w = [w9, w10, w11, w12, w13, w14]

vision_weight = 10

# For each Company row, generate score by weights determined above.
dfv = dfv[['Q9_A', 'Q9_B', 'Q9_C', 'Q9_D', 'Q9_E', 'Q9_F', 'Q9_G', 'Q9_H', 'Q9_I', 'Q10_A', 'Q10_B', 'Q10_C', 'Q10_D', 'Q10_E', 
           'Q11_A', 'Q11_B', 'Q11_C', 'Q11_D', 'Q11_E', 'Q11_F', 'Q11_G', 'Q12_A', 'Q12_B', 'Q12_C', 'Q12_D', 'Q12_E', 'Q12_F', 'Q12_G',
           'Q13_A', 'Q13_B', 'Q13_C', 'Q14', 'Q14_Other', 'Q15_A', 'Q15_B', 'Q15_C', 'Q15_D', 'Q15_E', 'Q15_F', 'Q15_G']].copy()

# Rows 6 (Q11), 7 (Q12), and 13 (Q11) respectively include 'No Response' in their columns, which we will recode to 'NaN'
dfv = dfv.replace(['No Response'], "NaN")
    
dfv

# Function to apply to each row:
def compute_score2(row):
    
    # Q9:
    # Choose all that apply: Sum
    
    s9 = 0
    categories = 9
    point9 = 0.2
    
    q9a = row['Q9_A']
    q9b = row['Q9_B']
    q9c = row['Q9_C']
    q9d = row['Q9_D']
    q9e = row['Q9_E']
    q9f = row['Q9_F']
    q9g = row['Q9_G']
    q9h = row['Q9_H']
    q9i = row['Q9_I']
  
    if q9a == "N/A, No EDI statement in place" or q9a == "NaN":
        s9 += 0
    if q9b == "Women and people who identify as women":
        s9 += point9
    if q9c == "Working mothers":
        s9 += point9
    if q9d == "Women of color, including all races and ethnicities":
        s9 += point9
    if q9e == "Women of all types of abilities":
        s9 += point9
    if q9f == "Persons of all non-binary gender identities": 
        s9 += point9
    if q9g == "Women of all sexual orientations":
        s9 += point9
    if q9h == "Women of all religions":
        s9 += point9
    if q9i == "Women of all socio-economic levels":
        s9 += point9
    
    if s9 > 1:
        s9 = 1
        
    # Q10: 
    # Conducted a demographic analysis of the current workforce = 20%
    # Conducted a baseline culture analysis using employee feedback to identify gaps = 20%
    # Shared performance on these analyses with all people managers = 20% (?)
    # Established goals regarding gender equity for the whole organization = 20%
    # Developed a timebound roadmap to meet established goals = 20%
    # None = 0% (q10_e excluded, since responses are either NaN or None)
    # Q11_e, _f: description / other / None, do not include in score
    # Sum, no cap

    s10 = 0
    point10 = 0.2
    
    q10a = row['Q10_A']
    q10b = row['Q10_B']
    q10c = row['Q10_C']
    q10d = row['Q10_D']
    
    if q10a == "Conducted a demographic analysis of the current workforce":
        s10 += point10
    if q10b == "Conducted a baseline culture analysis using employee feedback to identify gaps":
        s10 += point10
    if q10c == "Established goals regarding gender equity for the whole organization":
        s10 += point10
    if q10d == "Developed a timebound roadmap to meet established goals":
        s10 += point10
        
    # Q11:
    # Dedicated team responsible for establishing and tracking progress toward gender equity targets = 25%
    # Specific timeline on when targets should be met = 25%
    # Engagement with all staff on targets and their importance to the organization = 25%
    # Training of all employees who have a direct impact on meeting targets = 25%
    # No = 0%
    # Sum, no cap
    
    s11 = 0
    point11 = 0.25
    
    q11a = row['Q11_A']
    q11b = row['Q11_B']
    q11c = row['Q11_C']
    q11d = row['Q11_D']
    
    if q11a == "Dedicated team responsible for establishing and tracking progress toward gender equity targets":
        s11 += point11
    if q11b == "Specific timeline on when targets should be met":
        s11 += point11
    if q11c =="Engagement with all staff on targets and their importance to the organization":
        s11 += point11
    if q11d == "Training of all employees who have a direct impact on meeting targets":
        s11 += point11
    
    # Q12:
    # All Hiring Managers = 50%
    # Human Resources Dept = 50%
    # All Senior Management = 50%
    # C-Suite = 50%
    # N/A or No = 0%
    # Sum, cap at 100%
    
    s12 = 0
    point12 = 0.5
    
    q12a = row['Q12_A']
    q12b = row['Q12_B']
    q12c = row['Q12_C']
    q12d = row['Q12_D']
    
    if q12a == "All Hiring Managers":
        s12 += point12
    if q12b == "Human Resources Dept":
        s12 += point12
    if q12c == "All Senior Management": 
        s12 += point12
    if q12d == "C-Suite":
        s12 += point12
        
    if s12 > 1:
        s12 = 1
        
    # Q13:
    # No = 0%
    # Yes, internal statement made to all employees = 50%
    # Yes, official public/external statement made about organization's commitment to gender equity = 100%
    # Sum, cap at 100%
    
    s13 = 0
    
    #Q13_A: No => skip other scoring for this question
    q13b = row['Q13_B']
    q13c = row['Q13_C']
    
    if q13b == "Yes, internal statement made to all employees":
        s13 += 0.5
    if q13c == "Yes, official public/external statement made about organization's commitment to gender equity":
        s13 += 1
        
    if s13 > 1:
        s13 = 1
    
    # Q14
    # No EDI report is published = 0%
    # Publish an annual EDI report that includes performance data on gender equity = 33%
    # Publish an annual EDI report that includes performance data on gender equity with targets for future performance = 66%
    # Publish an annual EDI report that includes performance data on gender equity with targets for future performance and performance on previously set targets = 100%
    # Single pick (No Sum)
    s14 = 0
    
    q14 = row['Q14']
    
    if q14 == "Publish an annual EDI report that includes performance data on gender equity":
        s14 = 0.33
    if q14 == "Publish an annual EDI report that includes performance data on gender equity with targets for future performance":
        s14 = 0.66
    if q14 == "Publish an annual EDI report that includes performance data on gender equity with targets for future performance and performance on previously set targets":
        s14 = 1
    
    #preliminary score
    s = [s9, s10, s11, s12, s13, s14]
    
    # return (s9 * w9) + (s10 * w10) + (s11 * w11) + (s12 * w12) + (s13 * w13) + (s14 * w14)
    return sum(map(lambda a, b: a * b, s, w))

dfv.loc[:,'vision_prelim_score'] = dfv.apply(compute_score2, axis = 1)
dfv.loc[:,'vision_weighted_score'] = dfv['vision_prelim_score'] * vision_weight

dfv

,Q9_A,Q9_B,Q9_C,Q9_D,Q9_E,Q9_F,Q9_G,Q9_H,Q9_I,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11_A,Q11_B,Q11_C,Q11_D,Q11_E,Q11_F,Q11_G,Q12_A,Q12_B,Q12_C,Q12_D,Q12_E,Q12_F,Q12_G,Q13_A,Q13_B,Q13_C,Q14,Q14_Other,Q15_A,Q15_B,Q15_C,Q15_D,Q15_E,Q15_F,Q15_G,vision_prelim_score,vision_weighted_score
0,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,"Yes, internal statement made to all employees","Yes, official public/external statement made about organization's commitment to gender equity",No EDI report is published,NaN,Womens Empowerment Principles (UN),NaN,NaN,NaN,NaN,NaN,Gender Equity Now,0.380,2.8500
1,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,All Hiring Managers,Human Resources Dept,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.460,3.4500
2,NaN,NaN,NaN,"Women of color, including all races and ethnicities",NaN,NaN,NaN,NaN,NaN,NaN,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,Training of all employees who have a direct impact on meeting targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,No,NaN,NaN,Other,"Prepare report on work on EDI, no gender performance goals specifically called out.",NaN,NaN,NaN,NaN,NaN,None,NaN,0.150,1.1250
3,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conducted a demographic analysis of the current workforce,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.060,0.4500
4,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human Resources Dept,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.205,1.5375
5,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,No,NaN,NaN,Publish an annual EDI report that includes performance data on gender equity,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.083,0.6225
6,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Engagement with SOME staff on targets and their importance to the organization (at the leadership level- the teachers are mostly female),NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,"Yes, internal statement made to all employees",NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.130,0.9750
7,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conducted a demographic analysis of the current workforce,Conducted a baseline culture analysis using

Recruitment Category: Scoring
Recruitment Weight: 15

In [7]:
dfr = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/recruit_clean_final.csv", index_col=0)
dfr

,Organization,Q1,Q2,Q3,Q4_A,Q4_B,Q4_C,Q4_D,Q4_E,Q4_F,Q4_G,Q4_H,Q4_I,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q13_A,Q13_B,Q13_C1,Q13_C2,Q13_C3,Q13_C4,Q13_C5,Q13_C6,Q13_G7,Q13_C8,Q13_C9,Q13_J,Q14,Q15,Q17,Q18,Q19,Q20_A,Q20_B,Q20_C,Q20_D,Q21,benchmark_group
0,Company 1A,Yes,No low-diversity teams were identified.,Process has been designed and implemented,Statement about gender equity goals in recruitment on all job postings,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,"Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences","Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,NaN,NaN,NaN,NaN,In progress,N/A - No targets set,No,In progress of designing and implementing,NaN,In progress,N/A - No anti-bias performance review plan in place yet,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,N/A - No targets were established,In development,0.0,0.0,0.0,At the secondary or tertiary education level,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.0,Finance
1,Company 1B,Yes,A hiring and promotion plan is in place to address the low-diversity teams that have been found within the organization,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,NaN,"Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,"Yes, with all cohorts having a diverse population",No,In progress,No,N/A - No anti-bias performance review plan in place yet,7.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Women who identify as LGBTQ,NaN,NaN,N/A - No targets were established,No,NaN,NaN,NaN,NaN,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.0,Finance
2,Company 1C,No,N/A - No review conducted yet,No process has been designed yet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,"Yes, with many but not all cohorts having a diverse population",No,No,No,N/A - No anti-bias performance review plan in place yet,6.0,6.0,NaN,NaN,NaN,NaN,NaN,Women who identify as White/Caucasian,NaN,NaN,NaN,NaN,N/A - No targets were established,No,NaN,NaN,NaN,NaN,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.0,NaN
3,Company 1D,Yes,No low-diversity teams were identified.,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,"Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences","Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,Training for HR and all hiring managers on how to identify and reduce gender equity biases throughout the recruitment process,Oversight and support by trained HR or other designated-internal teams to ensure unbiased recruitment process,NaN,NaN,No,N/A - No targets set,No,No,No,No,N/A - No anti-bias performance review plan in place yet,20.0,80.0,NaN,NaN,NaN,NaN,NaN,Women who identify as White/Caucasian,Women who identify as Disabled,Women who identify as LGBTQ,NaN,NaN,N/A - No targets were established,No,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Technology
4,Company 1E,In progress,N/A - No review conducted yet,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure

In [8]:
w1 = 0.10
w2 = 0.05
w3 = 0.05
w4 = 0.10
w5 = 0.10
w6 = 0.05
w8 = 0.05
w9 = 0.10
w10 = 0.05
w11 = 0.10
w14 = 0.10
w15 = 0.05
w20 = 0.10
w = [w1, w2, w3, w4, w5, w6, w8, w9, w10, w11, w14, w15, w20]

recruit_weight = 15

# For each Company row, generate score by weights determined above.
dfr = dfr[['Q1', 'Q2', 'Q3', 'Q4_A', 'Q4_B', 'Q4_C', 'Q4_D', 'Q4_E', 'Q4_F', 'Q4_G', 'Q4_H', 'Q5', 'Q6', 'Q8', 'Q9', 'Q10', 'Q11', 'Q14', 'Q15', 'Q20_A', 'Q20_B', 'Q20_C', 'Q20_D']].copy()

# Rows including 'Other' display None. Change to NaN for consistency.
dfr = dfr.replace(['None'], "NaN")
    
dfr

# Function to apply to each row:
def compute_score3(row):
     
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    s5 = 0
    s6 = 0
    s8 = 0
    s9 = 0
    s10 = 0
    s11 = 0
    s14 = 0
    s15 = 0
    s20 = 0
    
    # Q1 / Q2:
    # No Sum / No Sum

    q1 = row['Q1']
    q2 = row['Q2']
    

    if q1 == "Yes":
        s2 = 1.0
    
        if q2 == "N/A (no review conducted yet)" or q2 == "In progress of determining":
            s2 = 0
        if q2 == "A hiring and promotion plan is in place to address the low-diversity teams that have been found within the organization" \
            or q2 == "No low-diversity teams were identified.":
            s2 = 1.0
        if q2 == "Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group)":
            s2 = 1.0
    else:
        if q1 == "No" or q1 == "NaN":
            s1 = 0
        if q1 == "In progress":
            s1 = 0.5

    # Q3 / Q4: 
    # No Sum / Sum, cap at 100%
    
    q3 = row['Q3']
    q4a = row['Q4_A']
    q4b = row['Q4_B']
    q4c = row['Q4_C']
    q4d = row['Q4_D']
    q4e = row['Q4_E']
    q4f = row['Q4_F']
    q4g = row['Q4_G']
    q4h = row['Q4_H']
    # Q4_I, J excluded since all values are NaN
    
    if q3 == "No process has been designed yet":
        s3 = 0
    else:
        if q3 == "Process has been designed, not yet implemented":
            s3 = 0.33
        if q3 == "Process has been designed and implemented":
            s3 = 1.0
            
        if q4a == "Statement about gender equity goals in recruitment on all job postings":
            s4 += 0.10
        if q4b == "Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated":
            s4 += 0.25
        if q4c == "Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences":
            s4 += 0.10
        if q4d == "Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunties are accessible to a diverse candidate pool":
            s4 += 0.25
        if q4e == "Blind resume review and blind auditions (as applicable)": 
            s4 += 0.25
        if q4f == "Training for HR and all hiring managers on how to identify and reduce gender equity biases throughout the recruitment process":
            s4 += 0.25
        if q4g == "Oversight and support by trained HR or other designated-internal teams to ensure unbiased recruitment process":
            s4 += 0.25
        if q4h == "Oversight by a third-party entity to identify and reduce bias from recruitment process":
            s4 += 0.25
            
        if s4 > 1:
            s4 = 1
        
    # Q5 / Q6:
    # No Sum / No Sum
    
    q5 = row['Q5']
    q6 = row['Q6']
    
    if q5 == "Yes":
        s5 = 1.0
        
        if q6 == "N/A - No targets set" or q6 == "N/A - No new hires made in the last 12 months":
            s6 = 0
        if q6 == "N/A - Plan in place for less than 12 months":
            s6 = 0.33
        if q6 == "Behind goals (<66% or fewer targets met)":
            s6 = 0.66
        if q6 == "Nearing goals (66-99% targets met)" or q6 == "Goals met" or q6 == "Goals exceeded":
            s6 = 1.0
            
    else:
        if q5 == "In progress":
            s5 = 0.5
        if q5 == "No":
            s5 = 0
    
    # Q8:
    # No Sum
    
    q8 = row['Q8']
    
    if q8 == "No":
        s8 = 0
    if q8 == "In progress of designing and implementing":
        s8 = 0.5
    if q8 == "Yes":
        s8 = 1.0
        
    # Q9:
    # No Sum
    
    q9 = row['Q9']
    
# Code for if score should be omitted entirely if NaN, instead of the 0 weighing the score down
#     if q9 == "NaN":
#         s.remove(s9)
#         w.remove(w9)

    if q8 == "No":
        s8 = 0
    if q8 == "In progress":
        s8 = 0.5
    if q8 == "Yes":
        s8 = 1.0
        
    # Q10:
    # No Sum

    q10 = row['Q10']
    
    if q10 == "N/A - No anti-bias performance review plan in place yet":
        s10 = 0
    if q10 == "No bias reduction":
        s10 = 0.33
    if q10 == "Modest bias reduction":
        s10 = 0.66
    if q10 == "Significant bias reduction":
        s10 = 1.0
        
        
    # Q11:
    # No Sum
    
    q11 = row['Q11']

    if q11 == "No":
        s11 = 0
    if q11 == "In progress":
        s11 = 0.5
    if q11 == "Yes":
        s11 = 1.0
    
    # Q14:
    # No Sum
    q14 = row['Q14']
    if q14 == "N/A - No targets were established":
        s14 = 0
    if q14 == "Behind goals (<66% or fewer targets met)":
        s14 = 0.5
    if q14 == "Nearing goals (66-99% targets met)" or q14 == "Goals met" or q14 == "Goals exceeded":
        s14 = 1.0
    
    # Q15:
    # No Sum
    q15 = row['Q15']
    if q15 == "No":
        s15 = 0 
    if q15 == "In development":
        s15 = 0.5
    if q15 == "Yes":
        s15 = 1.0
    
    # Q20: 
    # Multi-pick, cap at 100%
    
    q20a = row['Q20_A']
    q20b = row['Q20_B']
    q20c = row['Q20_C']
    q20d = row['Q20_D']

    if q20c != "No":     
        if q20a == "At the secondary or tertiary education level" \
        or q20b == "At the on-ramping/workforce re-entry level for mid-career women" \
        or q20d != "NaN":
            s20 = 1.0
    
    s = [s1, s2, s3, s4, s5, s6, s8, s9, s10, s11, s14, s15, s20]
    
    return sum(map(lambda a, b: a * b, s, w))

dfr.loc[:,'recruit_prelim_score'] = dfr.apply(compute_score3, axis = 1)
dfr.loc[:,'recruit_weighted_score'] = dfr['recruit_prelim_score'] * recruit_weight

dfr

,Q1,Q2,Q3,Q4_A,Q4_B,Q4_C,Q4_D,Q4_E,Q4_F,Q4_G,Q4_H,Q5,Q6,Q8,Q9,Q10,Q11,Q14,Q15,Q20_A,Q20_B,Q20_C,Q20_D,recruit_prelim_score,recruit_weighted_score
0,Yes,No low-diversity teams were identified.,Process has been designed and implemented,Statement about gender equity goals in recruitment on all job postings,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,"Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences","Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,NaN,NaN,NaN,In progress,N/A - No targets set,In progress of designing and implementing,NaN,In progress,N/A - No anti-bias performance review plan in place yet,N/A - No targets were established,In development,At the secondary or tertiary education level,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.3450,5.1750
1,Yes,A hiring and promotion plan is in place to address the low-diversity teams that have been found within the organization,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,NaN,"Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,NaN,NaN,NaN,No,N/A - No targets set,No,In progress,No,N/A - No anti-bias performance review plan in place yet,N/A - No targets were established,No,NaN,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.2250,3.3750
2,No,N/A - No review conducted yet,No process has been designed yet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,No,No,No,N/A - No anti-bias performance review plan in place yet,N/A - No targets were established,No,NaN,At the on-ramping/workforce re-entry level for mid-career women,NaN,NaN,0.1000,1.5000
3,Yes,No low-diversity teams were identified.,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,"Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences","Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,Training for HR and all hiring managers on how to identify and reduce gender equity biases throughout the recruitment process,Oversight and support by trained HR or other designated-internal teams to ensure unbiased recruitment process,NaN,No,N/A - No targets set,No,No,No,N/A - No anti-bias performance review plan in place yet,N/A - No targets were established,No,NaN,NaN,No,NaN,0.1850,2.7750
4,In progress,N/A - No review conducted yet,Process has been designed and implemented,NaN,Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated,"Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences","Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunities are accessible to a diverse candidate pool",NaN,Training for HR and all hiring managers on how to identify and reduce gender equity biases throughout the recruitment process,Oversight and support by trained HR or other designated-internal teams to ensure unbiased recruitment process,NaN,Yes,N/A - Plan in place for less than 12 months,In progress of designing and implementing,In progress,No,N/A - N

Compensation & Pay Equity: Scoring
Compensation Weight: 15

In [3]:
dfc = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/compensation_clean_final.csv", index_col=0)
dfc

,Q21_A,Q21_B,Q21_C,Q21_D,Q21_None,Q21_Other,Q22,Q23,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_None,Q25_A,Q25_B,Q25_C,Q25_D,Q25_E,Q25_Other,Q26,Q27_A,Q27_B,Q27_C,Q27_D,Q27_None,Q27_E,Q28,Q29_A,Q29_B,Q29_C,Q29_D,Q29_E,Q29_F,Q29_G,Q29_H,Q29_I,Q29_J,Q29_None,Q30_A,Q30_B,Q30_C,Q30_D,Q30_E,Q30_F,Q31_A,Q31_B,Q31_C,Q31_D,Q31_E,Q32,Q33,Q34_A,Q34_B,Q34_C_A,Q34_C_B,Q34_C_C,Q34_C_D,Q34_C_E,Q34_C_F,Q34_C_G,Q34_C_H,Q34_C_I,Q34_C_None,Q34_D,Q35,Q36,Q37,Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately from performance reviews,"The organization performs a ""job leveling"" or compensation analysis at least every two years",NaN,NaN,Yes,In development,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensation adjustments.,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay 

In [ ]:
w23 = 0.15
w24 = 0.10
w25 = 0.10
w28 = 0.05
w29 = 0.10
w30 = 0.05
w31 = 0.10
w32 = 0.05
w33 = 0.05
w34 = 0.10
w35 = 0.05
w36 = 0.10

w = [w23, w24, w25, w28, w29, w30, w31, w32, w33, w34, w35, w36]

comp_weight = 15

# For each Company row, generate score by weights determined above.
dfc = dfr[['Q23', 
           'Q24_A', 'Q24_B', 'Q24_C', 'Q24_D', 'Q24_E', 'Q24_F', 'Q24_None', 
           'Q25_A', 'Q25_B', 'Q25_C', 'Q25_D', 'Q25_E', 'Q25_Other',
           'Q28',
           'Q29_A', 'Q29_B', 'Q29_C', 'Q29_D', 'Q29_E', 'Q29_F', 'Q29_G', 'Q29_H', 'Q29_I', 'Q29_J', 'Q29_None',
           'Q30_A', 'Q30_B', 'Q30_C', 'Q30_D', 'Q30_E', 'Q30_F',
           'Q31_A', 'Q31_B', 'Q31_C', 'Q31_D', 'Q31_E',
           'Q32',
           'Q33',
           'Q34_A', 'Q34_B', 'Q34_C_A', 'Q34_C_B', 'Q34_C_C', 'Q34_C_D', 'Q34_C_E', 'Q34_C_F', 
               'Q34_C_G', 'Q34_C_H', 'Q34_C_I 	Q34_C_None 	
           Q34_D 	Q35 	Q36 	Q37']].copy()

# Rows including 'Other' display None. Change to NaN for consistency.
dfr = dfr.replace(['None'], "NaN")
    
dfr

# Function to apply to each row:
def compute_score3(row):
     
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    s5 = 0
    s6 = 0
    s8 = 0
    s9 = 0
    s10 = 0
    s11 = 0
    s14 = 0
    s15 = 0
    s20 = 0
    
    # Q1 / Q2:
    # No Sum / No Sum

    q1 = row['Q1']
    q2 = row['Q2']
    

    if q1 == "Yes":
        s2 = 1.0
    
        if q2 == "N/A (no review conducted yet)" or q2 == "In progress of determining":
            s2 = 0
        if q2 == "A hiring and promotion plan is in place to address the low-diversity teams that have been found within the organization" \
            or q2 == "No low-diversity teams were identified.":
            s2 = 1.0
        if q2 == "Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group)":
            s2 = 1.0
    else:
        if q1 == "No" or q1 == "NaN":
            s1 = 0
        if q1 == "In progress":
            s1 = 0.5

    # Q3 / Q4: 
    # No Sum / Sum, cap at 100%
    
    q3 = row['Q3']
    q4a = row['Q4_A']
    q4b = row['Q4_B']
    q4c = row['Q4_C']
    q4d = row['Q4_D']
    q4e = row['Q4_E']
    q4f = row['Q4_F']
    q4g = row['Q4_G']
    q4h = row['Q4_H']
    # Q4_I, J excluded since all values are NaN
    
    if q3 == "No process has been designed yet":
        s3 = 0
    else:
        if q3 == "Process has been designed, not yet implemented":
            s3 = 0.33
        if q3 == "Process has been designed and implemented":
            s3 = 1.0
            
        if q4a == "Statement about gender equity goals in recruitment on all job postings":
            s4 += 0.10
        if q4b == "Process in-place (with designated and trained teammates or usage of technology) to review and revise job descriptions prior to posting to ensure gender-biased language is eliminated":
            s4 += 0.25
        if q4c == "Move toward skills-based competencies and growth mindsets and away from focus on certain schools, prior experiences":
            s4 += 0.10
        if q4d == "Requirement to post jobs on a diverse group of job boards, recruiting firms, etc to ensure opportunties are accessible to a diverse candidate pool":
            s4 += 0.25
        if q4e == "Blind resume review and blind auditions (as applicable)": 
            s4 += 0.25
        if q4f == "Training for HR and all hiring managers on how to identify and reduce gender equity biases throughout the recruitment process":
            s4 += 0.25
        if q4g == "Oversight and support by trained HR or other designated-internal teams to ensure unbiased recruitment process":
            s4 += 0.25
        if q4h == "Oversight by a third-party entity to identify and reduce bias from recruitment process":
            s4 += 0.25
            
        if s4 > 1:
            s4 = 1
        
    # Q5 / Q6:
    # No Sum / No Sum
    
    q5 = row['Q5']
    q6 = row['Q6']
    
    if q5 == "Yes":
        s5 = 1.0
        
        if q6 == "N/A - No targets set" or q6 == "N/A - No new hires made in the last 12 months":
            s6 = 0
        if q6 == "N/A - Plan in place for less than 12 months":
            s6 = 0.33
        if q6 == "Behind goals (<66% or fewer targets met)":
            s6 = 0.66
        if q6 == "Nearing goals (66-99% targets met)" or q6 == "Goals met" or q6 == "Goals exceeded":
            s6 = 1.0
            
    else:
        if q5 == "In progress":
            s5 = 0.5
        if q5 == "No":
            s5 = 0
    
    # Q8:
    # No Sum
    
    q8 = row['Q8']
    
    if q8 == "No":
        s8 = 0
    if q8 == "In progress of designing and implementing":
        s8 = 0.5
    if q8 == "Yes":
        s8 = 1.0
        
    # Q9:
    # No Sum
    
    q9 = row['Q9']
    
# Code for if score should be omitted entirely if NaN, instead of the 0 weighing the score down
#     if q9 == "NaN":
#         s.remove(s9)
#         w.remove(w9)

    if q8 == "No":
        s8 = 0
    if q8 == "In progress":
        s8 = 0.5
    if q8 == "Yes":
        s8 = 1.0
        
    # Q10:
    # No Sum

    q10 = row['Q10']
    
    if q10 == "N/A - No anti-bias performance review plan in place yet":
        s10 = 0
    if q10 == "No bias reduction":
        s10 = 0.33
    if q10 == "Modest bias reduction":
        s10 = 0.66
    if q10 == "Significant bias reduction":
        s10 = 1.0
        
        
    # Q11:
    # No Sum
    
    q11 = row['Q11']

    if q11 == "No":
        s11 = 0
    if q11 == "In progress":
        s11 = 0.5
    if q11 == "Yes":
        s11 = 1.0
    
    # Q14:
    # No Sum
    q14 = row['Q14']
    if q14 == "N/A - No targets were established":
        s14 = 0
    if q14 == "Behind goals (<66% or fewer targets met)":
        s14 = 0.5
    if q14 == "Nearing goals (66-99% targets met)" or q14 == "Goals met" or q14 == "Goals exceeded":
        s14 = 1.0
    
    # Q15:
    # No Sum
    q15 = row['Q15']
    if q15 == "No":
        s15 = 0 
    if q15 == "In development":
        s15 = 0.5
    if q15 == "Yes":
        s15 = 1.0
    
    # Q20: 
    # Multi-pick, cap at 100%
    
    q20a = row['Q20_A']
    q20b = row['Q20_B']
    q20c = row['Q20_C']
    q20d = row['Q20_D']

    if q20c != "No":     
        if q20a == "At the secondary or tertiary education level" \
        or q20b == "At the on-ramping/workforce re-entry level for mid-career women" \
        or q20d != "NaN":
            s20 = 1.0
    
    s = [s1, s2, s3, s4, s5, s6, s8, s9, s10, s11, s14, s15, s20]
    
    return sum(map(lambda a, b: a * b, s, w))

dfr.loc[:,'recruit_prelim_score'] = dfr.apply(compute_score3, axis = 1)
dfr.loc[:,'recruit_weighted_score'] = dfr['recruit_prelim_score'] * recruit_weight

dfr